In [1]:
import os
HOME = os.getcwd()
print(HOME)

c:\Users\WilhelmB\Documents\DiagAssistAI


In [3]:
%pip install -e .

Note: you may need to restart the kernel to use updated packages.


In [1]:
from src.LangSAM.SamLangDino import SamDino
from src.DataSets.Mamitas_Thermal_Dataset.Mamitas_Dataset import Mamitas_Thermal_Feet_Dataset, ToBoolTensor, PermuteTensor 
from torchvision.transforms import transforms
import torch

c:\Users\WilhelmB\miniconda3\envs\DiagIA\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\users\wilhelmb\documents\diagassistai\src\groundingdino\models\GroundingDINO\ms_deform_attn.py:31: UserWarning: Failed to load custom C++ ops. Running on CPU mode Only!
  warnings.warn("Failed to load custom C++ ops. Running on CPU mode Only!")


In [2]:
model = SamDino(SAM='vit_h')

AttributeError: 'tuple' object has no attribute 'tb_frame'

In [4]:
transform_mask = transforms.Compose([
    ToBoolTensor(),
    PermuteTensor((1,2,0))
])
transform_img = transforms.Compose([
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.uint8),
    PermuteTensor((1,2,0))
])

_, val_dataset = Mamitas_Thermal_Feet_Dataset().generate_dataset_with_val(torch_dataset=True,
                                                   batch_size=38,
                                                   shuffle=False,
                                                   split_val=0.2,
                                                   merge_image=True,
                                                   transform_mask=transform_mask,
                                                   transform_img=transform_img)


Dataset URL: https://www.kaggle.com/datasets/lucasiturriago/mamitas-thermal-feet
Train_dataset: 150
Val_dataset: 38


In [5]:
text_prompt="feet,foot"
box_threshold=0.10
text_threshold=0.25
process_box_threshold = 0.10

boxes,logits,phrases,images,ids,masks_o,masks_p = model.LangSam_batch(image_data=val_dataset,
                                                                      text_prompt=text_prompt,
                                                                      box_threshold=box_threshold,
                                                                      process_box_threshold=process_box_threshold,
                                                                      text_threshold=text_threshold,
                                                                      mode_predict='box',
                                                                      mode_data='batch')

NameError: name '_C' is not defined